# Загрузка, препроцессинг и запись слов песен


In [24]:
#!pip install pymorphy2
#!pip install lyricsgenius
#!pip install --user -U nltk

In [14]:
import lyricsgenius
import csv
import re
import time
import pymorphy2

### Подключение к Genius

Для использования библиотеки lyricsgenius нужно получить токен API Genius. Как его получить описано в  [документации](https://docs.genius.com/).

In [2]:
# token = 'YOUR_TOKEN_HERE'
genius = lyricsgenius.Genius(token)
genius.remove_section_headers = True

### Парсинг csv с названиями песен

In [3]:
def parse_songs(data):
    with open(data) as csv_data:
        songs = csv.reader(csv_data, delimiter=',')
        songs_list = list(songs)
    return songs_list[1:]

Рассмотрим данные плейлиста двухтысячных

In [4]:
data_00 = 'data/data_00.csv'

In [5]:
songs_00 = parse_songs(data_00)

In [6]:
songs_00[:10] # первые 10 песен

[['Реви', 'Иванушки International'],
 ['Disco Superstar', 'Дискотека Авария'],
 ['Школа № 7', 'Hi-Fi'],
 ['Ты где-то', 'Гости из будущего'],
 ['Лето-зима', 'Татьяна Буланова'],
 ['Беспризорник', 'Hi-Fi'],
 ['Беги, беги', 'Иванушки International'],
 ['Люби меня по-французски', 'Гости из будущего'],
 ['Ласковая моя', 'Чай вдвоём'],
 ['Девушки как звезды', 'Андрей Губин']]

### Получение текстов песен

In [7]:
def get_lyrics(song_data):
    artist = genius.search_artist(song_data[1], max_songs=1, sort='title')
    message = 'no lyrics found'
    
    if artist is None:
        return message
    
    else:
        song = genius.search_song(song_data[0], artist.name)
    
        if song is None:
            return message
    
        lyrics = song.lyrics
        return lyrics

Для примера попробуем получить текст первой песни из плейлиста

In [8]:
lyrics1 = get_lyrics(songs_00[0])

Searching for songs by Иванушки International...

Changing artist name to 'Иванушки International (Ivanushki International)'
Song 1: "Алешкина Любовь (Aleshkina Love)"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Реви" by Иванушки International (Ivanushki International)...
Done.


Сырой текст до препроцессинга:

In [9]:
lyrics1

'Не кляни ты меня\nВ жадных каплях дождя\nЧто не так я всё видел как есть\nПотому что\u2005всегда\nЯ\u2005искал для тебя\nЧего\u2005нет ни там и ни здесь\nКлёны\u2005шепчут: люби\nА берёзы: целуй\nИ ты время сожмёшь в кулачок\nВ уголочке\u205fнебес\u205fплотик\u205fиз облаков\nУнесёт нас\u205fна обрывы веков\nПрипев:\nА\u205fты не сдерживай слёз — реви, реви\nВедь я не всерьёз в твоей любви\nА ты не сдерживай слёз — реви, реви\nКак птицу, любовь корми с руки\nА ты не сдерживай слёз...\nА ты не сдерживай слёз...\nЭто всё горизонт\nНе подвластен нам он\nКогда с "Вы" переходишь на "Ты"\nЭто клён-почтальон\nЗаблудился как сон\nБандерольку принёс от Луны\nПрипев:\nА ты не сдерживай слёз — реви, реви\nВедь я не всерьёз в твоей любви\nА ты не сдерживай слёз — реви, реви\nКак птицу, любовь корми с руки\nА ты не сдерживай слёз...\nА ты не сдерживай слёз...\nА ты не сдерживай слёз — люби, люби\nА ты не сдерживай слёз — люби, люби\nКак птицу, любовь корми с руки\nА ты не сдерживай слёз — реви, ре

### Препроцессинг и лемматизация

Делаем с текстом следующее:
* приводим к нижнему регистру;
* удаляем стоп-слова;
* удаляем знаки препинания и прочие символы;
* производим лемматиацию - приведение слов к начальным формам


Список стоп-слов русского языка взят из библиотеки [nltk](https://www.nltk.org/)

Лемматизация произвоится с помощью морфологического анализатора [pymorphy2](https://pymorphy2.readthedocs.io/)

In [15]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize(song):
    
    doc = song.split()
    tokens = ""
    
    for i in range(len(doc)):
        token = morph.normal_forms(doc[i])[0]
        tokens = tokens + token + " "
    return(tokens)

In [23]:
from nltk.corpus import stopwords

stopwords = set(stopwords.words('russian'))
custom_stopwords = ['припев', 'куплет', 'интро']  # дополним список стоп-слов
stopwords.update(custom_stopwords)

def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower().split()
    text = [word for word in text if word not in stopwords]
    text = ' '.join(text)
    text = lemmatize(text)
    return text

Текст песни после препроцессинга:

In [17]:
clean_lyrics1 = preprocess(lyrics1)

In [19]:
clean_lyrics1

'клясть жадный капля дождь всё видеть искать клён шептать любить берёза целовать время сжать кулачок уголочек небо плотик облако унести обрыв век сдерживать слеза реветь реветь всерьёз твой любовь сдерживать слеза реветь реветь птица любовь кормить рука сдерживать слеза сдерживать слеза это всё горизонт подвластный мы переходить это клёнпочтальон заблудиться сон бандеролька принести луна сдерживать слеза реветь реветь всерьёз твой любовь сдерживать слеза реветь реветь птица любовь кормить рука сдерживать слеза сдерживать слеза сдерживать слеза любить любить сдерживать слеза любить любить птица любовь кормить рука сдерживать слеза реветь реветь всерьёз твой любовь сдерживать слеза реветь реветь птица любовь кормить рука сдерживать слеза сдерживать слеза реветь реветь птица любовь кормить рука '

### Запись текстов в csv файл

Функция принимает список песен и записывает csv с найденными обработанными текстами. Выводит на экран песни, которые не нашлись на genius.com.

In [20]:
def write_lyrics(song_list, csv_name):
    counter = 0
    not_found = []
    
    with open(csv_name,'w') as out:
        csv_out=csv.writer(out)
        
        for row in song_list:
            lyrics = get_lyrics(row)
            if lyrics == 'no lyrics found':
                not_found.append(row)
                continue
                
            lyrics = preprocess(lyrics)
            counter += 1
            csv_row = (lyrics, counter)
            csv_out.writerow(csv_row)
            print('sleeeping for good parse')
            time.sleep(5)
            
    print('Найдено: ', counter)
    print('Не найдены песни: ')
    for song in not_found:
        print(song)

In [21]:
file_00 = 'lyrics_00.csv' # название нового файла для записи обработанных песен

In [62]:
write_lyrics(songs_00, file_00)

Searching for songs by Иванушки International...

Changing artist name to 'Иванушки International (Ivanushki International)'
Song 1: "Алешкина Любовь (Aleshkina Love)"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Реви" by Иванушки International (Ivanushki International)...
Done.
sleeeping for good parse
Searching for songs by Дискотека Авария...

Changing artist name to 'Дискотека Авария (Diskoteka Avaria)'
Song 1: "Коуч (Kouch)"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Disco Superstar" by Дискотека Авария (Diskoteka Avaria)...
No results found for: 'Disco Superstar Дискотека Авария (Diskoteka Avaria)'
Searching for songs by Hi-Fi...

Changing artist name to 'Czarny HIFI'
Song 1: "Anti-Hype"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Школа № 7" by Czarny HIFI...
No results found for: 'Школа № 7 Czarny HIFI'
Searching for songs by Гости из будущего...

Changing artist name to 'Гости из буд

Changing artist name to 'Филипп Киркоров (Philipp Kirkorov)'
Song 1: "Ba Mir Bistu Shein"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Жестокая любовь" by Филипп Киркоров (Philipp Kirkorov)...
Done.
sleeeping for good parse
Searching for songs by ВИА ГРА...

Changing artist name to 'ВИА Гра (Nu Virgos)'
Song 1: "1+1"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Попытка № 5" by ВИА Гра (Nu Virgos)...
Done.
sleeeping for good parse
Searching for songs by Чай вдвоём...

Changing artist name to 'Чай Вдвоем (Chay Vdvoem)'
Song 1: "24 Часа (24 Hours)"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Желанная" by Чай Вдвоем (Chay Vdvoem)...
Done.
sleeeping for good parse
Searching for songs by Руки Вверх!...

Changing artist name to 'Руки Вверх (Ruki Vverh)'
Song 1: "18 мне уже (18 mne uzhe)"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Ай-яй-яй" by Руки Вверх (Ruki Vverh)..

No results found for: 'Выше облаков ВИА Сливки (VIA Slivki)'
Searching for songs by Валерия...

Changing artist name to 'Валерия (Valeriya)'
Song 1: "Back to Love"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Не обижай меня" by Валерия (Valeriya)...
Done.
sleeeping for good parse
Searching for songs by Блестящие...

Changing artist name to 'Блестящие (Blestyashchiye)'
Song 1: "А я всё летала (And I flew all the way)"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "За четыре моря" by Блестящие (Blestyashchiye)...
No results found for: 'За четыре моря Блестящие (Blestyashchiye)'
Searching for songs by REFLEX...

Changing artist name to 'REFLEX (RU)'
Song 1: "Non Stop"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Non Stop" by REFLEX (RU)...
Done.
sleeeping for good parse
Searching for songs by Алла Пугачёва...

Changing artist name to 'Алла Пугачёва (Alla Pugacheva)'
Song 1: "22+28"

Reached user-spe

Done.
sleeeping for good parse
Searching for songs by Юлия Савичева...

Changing artist name to 'Юлия Савичева (Yulia Savicheva)'
Song 1: "Believe Me"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Отпусти" by Юлия Савичева (Yulia Savicheva)...
Done.
sleeeping for good parse
Searching for songs by НЕПАРА...

No results found for 'НЕПАРА'.
Searching for songs by НЕПАРА...

No results found for 'НЕПАРА'.
Searching for songs by Иракли...

Changing artist name to 'Иракли (Irakli)'
Song 1: "Вова-чума (Vova-chuma)"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Вова-чума" by Иракли (Irakli)...
Done.
sleeeping for good parse
Searching for songs by Мария Ржевская...

Changing artist name to 'Мария Ржевская (Maria Rzhevskaya)'
Song 1: "Когда я стану кошкой (English translation)"

Reached user-specified song limit (1).
Done. Found 1 songs.
Searching for "Когда я стану кошкой" by Мария Ржевская (Maria Rzhevskaya)...
Done.
sleeeping for goo